# modifyParameters

- This notebook is designed to open the default CLM parameter file and modify one or several fields.  
- It's easier to run if you've made a local copy of the parameter file somewhere in your working directory.
- You can find the parameter file by looking at your `lnd_in` file from a case directory on the command line.
    > e.g. `cat ~/scratch/NEON_cases/KONZ.transient/run/lnd_in | grep paramfile`
- Then I created a copy of this parameter file here (~/scratch/NEON_cases/.)
    > on Cheyenne: `cp /glade/p/cesmdata/cseg/inputdata/lnd/clm2/paramdata/ctsm51_params.c211112.nc ~/scratch/NEON_cases/.`
    
This tutorial has 3 parts:
1. Open the parameter file
2. Modify a global parameter
3. Modify a pft specific parameter
Created by Will Wieder Nov 2021
Modified for NEON example May 2023

In [ ]:
# load libraries
import xarray as xr
import pandas as pd
import os
import netCDF4
import numpy as np

## 1. Open the parameter file
As mentioned above, I've alreay copied the file to a local directory.

In [ ]:
pathin = '~/scratch/NEON_cases/'
basefile = pathin + 'ctsm51_params.c211112.nc'
p = xr.open_dataset(basefile,decode_times=False) 
p.var

### CLM has > 400 parameters!
It's too much for us to go over here. Hopefully between the [user's guide and tech note](https://escomp.github.io/ctsm-docs/versions/master/html/index.html) in combination with the [model code](https://escomp.github.html) you can find what you need to regarding parameters.

For now, we'll just look at how to change a few parameters as an example.

You'll notice some parameters are dimensioned by plant functional type (pft) while others are global.  The global parameters are easier to modify, so we'll to that first.

## 2. Changing global parameters
For example, the parameter `rf_cwdl2` is used in the soil biogeogeochemistry code.
- Can you find out what the default values for this parameter is?
- Can you understand what it controls in the model?
- Can you find where it's used in the model code?

In [ ]:
p.rf_cwdl2

Let's imagine that you studied wood decomposition at a NEON site and have found that a 75% of wood decomposition is respired to the atmosphere. We can create a new parameter file, change the values for `rf_cwdl2`, and save the new file.

In [ ]:
p2 = p.copy(deep=True)       # create a copy of the parameter file to modify
p2['rf_cwdl2'].values = 0.75 # change the values of our parameter
p2.rf_cwdl2                  # check that it worked as intended

In [ ]:
# now save the file
p2.to_netcdf(pathin + 'ctsm51_params.c211112_high_rf_CWD.nc')

You'll notice this is a non-standard way to modify parameter files in CTSM.  It's really just designed for quick experiments:
- I like to make the name descriptive so I remember what I changed.
- One disadvantage here is that we didn't append the attribues of the .nc file to record this change.
- There are lots of other more efficient ways to do this for parameter perturbation experiments, that we'll eventually cover in future tutorials.

## 3. Modify a pft specific parameter
For this example, let's assume you looked at leaf traits that were measured at for your favorite NEON site and realized that the values being used in CLM in global simulations are not representative of local site conditions.  Here we can do this for temperate deciduous forests at Steigerwald (STEI).  

<div class="alert alert-block alert-warning">

<b>HINT:</b>  It's a good idea to check what PFT is actually on the surface dataset for the site you're running.

</div>

Let's see how PFTs are named in the parameter file, this will help us change values for particular PFTs more accurately

In [ ]:
p.pftname.values[0:16]

In [ ]:
# now we'll create an index for this PFT and print the foliar C:N ratio that's used in CLM

In [ ]:
mypft = 'broadleaf_deciduous_temperate_tree'
ix    = np.array([mypft in str(n) for n in p.pftname.values])  #index vector for mypft
param = 'leafcn'
m = p[param].values
print(m[ix])

In [ ]:
# Now we can change this value with something that's hypothetically been observed.
new_leafcn = 30
p3 = p.copy(deep=True)
p3[param].values[ix]= new_leafcn
p3[param][0:16]

In [ ]:
# now save the file
p3.to_netcdf(pathin + 'ctsm51_params.c211112_tdf_leafcn30.nc')

---

<div class="alert alert-block alert-success">
<b>Congratulations:</b> 
    
You can now modify parameter files!  
Remember, to run with one of these you'll have to spin up the model first.

</div>
